**A Detailed Look at NFL Punts and Player Safety**

**Our View of the Initial Challenge**

* It’s impossible to completely eliminate punt related concussions and keep punting as a valuable part of the game.  Therefor rule changes are recommended to keep both the punts and fake punts part of the game while reducing concussions.  
* By looking at the challenge data and provided videos it was clear that player velocity in conjunction with returnable longer punts is leading to concussions. 
* How do you slow down the velocity and force of an impact from the punting teams initial lineman/blockers on both the punt and the pursuit of the punt returner, or reduce the number of returns?
    1. 20 of the 27 (74%) concussions from the punting team come from these 7 position players in blue in the image. 
    2. The velocity of the players in blue is greater due to the amount of time/ground they are allowed to run given their “free” break towards the punt returner.
    3. Only 2 concussions occurred on punts with a fair catch, so we ideally want more fair catches to reduce concussions. 

![Formation](https://i.imgur.com/qHoJYAL.png)

**Finding Correlation Between Time from the Play Snap to the Punt and Subsequent Punt Distance**

A cornerstone of our proposed rule change is that shorter punts produce less concussions.  We found several proofs of this in the data, which I will explain now:

1)  *Proof that longer punts correlate with more concussions*
The following JAVA code parses the punter and punt distances from the PlayDescription field and produces a clean CSV with aggregate totals for punt distances.



private static void parsePuntDesc() {
		try {
			BufferedReader br = new BufferedReader(new FileReader("<HOME>\\NFLPunts.txt"));
			PrintWriter pw = new PrintWriter("<HOME>\\parsePunts.txt");
			try {
			    String line = br.readLine();
			    
			    while (line != null) {
			    	String[] s = line.split(" ");
			    	int puntsToken = -1;
			    	for(int i=0;i<s.length-1;i++) {
			    		if(s[i].equals("punts")) {
			    			puntsToken = i;
			    		}
			    	}
			    	if(puntsToken==-1) {
			    		String punter = "NA";
				    	String yards = "NA";
				    	System.out.println("punter = " + punter + ", yards = " + yards);
				    	pw.println(punter + "\t" + yards);
			    	} else {
				    	String punter = s[puntsToken-1];
				    	String yards = s[puntsToken+1];
				    	System.out.println("punter = " + punter + ", yards = " + yards);
				    	pw.println(punter + "\t" + yards);
			    	}
			        line = br.readLine();
			    }
			    pw.close();
			} finally {
			    br.close();
			}
		} catch(Exception e){e.printStackTrace();}
	}

With the totals from the aggregate punt, we correlated the distance of punts that produced concussions and graphed the information using Excel.

 ![ConcurrentGraph](https://i.imgur.com/wHhLhUE.png)

As you can see, the trend observed by the smoothed moving average predicts a reasonably steady increase of concussive events as punt distances increase.

2)  *Proof that punting the ball more quickly after the play snap ultimately decreases total punt distance.*

For this, the data from NGS documents in the competition data were loaded into SQL Server.  For each PlayID and GameKey, the amount of time between the snap and the punt can be found with this query:

select distinct time as punt_time, lag(time) over (order by time) as snap_time, datediff(MILLISECOND, lag(time) over (order by time),time) as punt_to_snap_time, pinfo.gamekey, pinfo.playid, playdescription, event from [NGS-2016-reg-wk1-6] ngs, play_information pinfo where pinfo.GameKey = ngs.GameKey and pinfo.playid= ngs.PlayID and (event = 'ball_snap' or event = 'punt')  order by time desc

Also note, the below query was used to derive player velocity at any given moment:

select  time, event, x, lag(x) over (order by time) as previous_x, y, lag(y) over (order by time) as previous_y, SQRT(POWER(cast(x as float)-cast(lag(x) over (order by time) as float),2)+POWER(cast(y as float)-cast(lag(y) over (order by time) as float),2)) * 10.0 as yards_per_second from [NGS-2016-reg-wk1-6] 
where GameKey = 144 and PlayID = 2342 and GSISID = 23259 order by time desc

By utilizing the below code, I've extracted this into the attached "Time from snap to punt and distance" data attachment with this Kernel.  In this code, I also correlate the distance of the punts associated with the time before the punter kicks it.

public static void main(String[] args) {
		 try
         {
			 PrintWriter pw = new PrintWriter("c:\\jon\\timeAndDist.txt");
			 Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver");

             String userName = "sa";
             String password = "password";
             String url = "jdbc:sqlserver://localhost\\sqlexpress:1433;databaseName=NFL";
             Connection con = DriverManager.getConnection(url, userName, password);
             Statement s1 = con.createStatement();
             ResultSet rs = s1.executeQuery("select distinct time as punt_time, lag(time) over (order by time) as snap_time, datediff(MILLISECOND, lag(time) over (order by time),time) as punt_to_snap_time, pinfo.gamekey, pinfo.playid, playdescription, event from [NGS-2016-reg-wk1-6] ngs, play_information pinfo where pinfo.GameKey = ngs.GameKey and pinfo.playid= ngs.PlayID and (event = 'ball_snap' or event = 'punt')  order by time desc");
             
             if(rs!=null){
                 while (rs.next()){
                     double puntToSnapTime = rs.getDouble("punt_to_snap_time");
                     String playDesc = rs.getString("playdescription");
                     if(puntToSnapTime > 1000 && puntToSnapTime < 5000) {
                    	 String parsedDesc = parseDesc(playDesc);
                    	 if(!parsedDesc.contains("NA")) pw.println(parseDesc(playDesc) + "," + puntToSnapTime/1000.0 + "," + "5");
                     }

                 }
             }
             pw.close();
             //String result = new result[20];

         } catch (Exception e)
         {
             e.printStackTrace();
         }

Now you can run a feedforward regressive model on the aforementioned data to see that the quicker a punter kicks after the snap, the shorter the punt tends to be (it is not a smoothed line...it is really just that linear).  Below is the code for running this in DL4J.



public class PuntTimeToDistance {

    public static final int seed = 12345;
    public static final int nEpochs = 200;
    public static final int nSamples = 833;
    public static final int batchSize = 100;
    public static final double learningRate = 0.01;
    public static int MIN_RANGE = 0;
    public static int MAX_RANGE = 100;

    public static final Random rng = new Random(seed);

    public static void main(String[] args){

        //Generate the training data
        DataSetIterator iterator = getTrainingData(batchSize,rng);

        MultiLayerNetwork net = new MultiLayerNetwork(new NeuralNetConfiguration.Builder()
                .seed(seed)
                .weightInit(WeightInit.XAVIER)
                .updater(new Nesterovs(learningRate, 0.9))
                .list()
                .layer(0, new DenseLayer.Builder().nIn(2).nOut(50)
                        .activation(Activation.SIGMOID)
                        .build())
                .layer(1, new DenseLayer.Builder().nIn(50).nOut(5)
                        .activation(Activation.RELU)
                        .build())
                .layer(2, new OutputLayer.Builder(LossFunctions.LossFunction.MSE)
                        .activation(Activation.IDENTITY)
                        .nIn(5).nOut(1).build())
                .build()
        );
        net.init();
        net.setListeners(new ScoreIterationListener(1));


        //Train the network on the full data set, and evaluate in periodically
        for( int i=0; i<nEpochs; i++ ){
            iterator.reset();
            net.fit(iterator);
        }

        double[] xVal = new double[15];
        double[] yVal = new double[15];
        int counter = 0;
        
        for(double x=1.5; x<=3.0; x += .1) {
	        final INDArray input = Nd4j.create(new double[] { x,5 }, new int[] { 1,2 });
	        INDArray out = net.output(input, false);
	        System.out.println("At " + x + " seconds, yards = " + Double.parseDouble(out.toString())*10.0);
	        xVal[counter] = x;
	        yVal[counter] = Double.parseDouble(out.toString())*10.0;
	        counter++;
        }
        plot(xVal, yVal);
        

    }

    private static DataSetIterator getTrainingData(int batchSize, Random rand){
    	final String filenameTrain  = "<HOME>\\timeAndDist.txt";

    	double [] yards = new double[nSamples];
        double [] time = new double[nSamples];
        double [] offset = new double[nSamples];
        
        try {
        	BufferedReader br = new BufferedReader(new FileReader(filenameTrain));
		    String line = br.readLine();
		    int i = 0;
		    while (line != null) {
		    	String[] s = line.split(",");
		    	
		    	time[i] = Double.parseDouble( s[1]);
	            offset[i] =  Double.parseDouble( s[2]);
	            yards[i] = Double.parseDouble( s[0])/10.0;
		    	
		    	i++;
		    	line = br.readLine();
		    }
		    br.close();
        } catch(Exception e) {e.printStackTrace();}
       
        INDArray inputNDArray1 = Nd4j.create(time, new int[]{nSamples,1});
        INDArray inputNDArray2 = Nd4j.create(offset, new int[]{nSamples,1});
        INDArray inputNDArray = Nd4j.hstack(inputNDArray1,inputNDArray2);
        INDArray outPut = Nd4j.create(yards, new int[]{nSamples, 1});
        DataSet dataSet = new DataSet(inputNDArray, outPut);
        List<DataSet> listDs = dataSet.asList();
        Collections.shuffle(listDs,rng);
        return new ListDataSetIterator(listDs,batchSize);


    }
    
    private static DataSetIterator getTestData(int batchSize, Random rand){
    	final String filenameTrain  = "<HOME>\\timeAndDist.txt";

        //Load the training data:
        RecordReader rr = new CSVRecordReader();
        DataSetIterator trainIter = null;
        try {
        rr.initialize(new FileSplit(new File(filenameTrain)));
        trainIter = new RecordReaderDataSetIterator(rr,batchSize,0,2);
        } catch(Exception e) {e.printStackTrace();}
        return trainIter;

    }
    
    private static void plot(final double[] x, final double[] y) {
        final XYSeriesCollection dataSet = new XYSeriesCollection();
        final XYSeries s = new XYSeries("NGS Data");
        for(int i=0;i<x.length;i++) {
        	s.add(x[i], y[i]);
        }
        dataSet.addSeries(s);

        final JFreeChart chart = ChartFactory.createXYLineChart(
                "Punt Length by Time Before Punt",      // chart title
                "Time Between Snap and Punt",                        // x axis label
                "Predicted Distance", // y axis label
                dataSet,                    // data
                PlotOrientation.VERTICAL,
                true,                       // include legend
                true,                       // tooltips
                false                       // urls
        );

        XYPlot xyPlot = chart.getXYPlot();
        ValueAxis rangeAxis = xyPlot.getRangeAxis();
        rangeAxis.setRange(40.0,50.0);
        final ChartPanel panel = new ChartPanel(chart);
        
        XYItemRenderer renderer = xyPlot.getRenderer();
        BasicStroke stroke = new BasicStroke(3f);
        renderer.setSeriesStroke(0, stroke);

        final JFrame f = new JFrame();
        f.add(panel);
        f.setDefaultCloseOperation(WindowConstants.EXIT_ON_CLOSE);
        f.pack();

        f.setVisible(true);
    }
    
    
    private static void addSeries(final XYSeriesCollection dataSet, final INDArray x, final INDArray y, final String label){
        final double[] xd = x.data().asDouble();
        final double[] yd = y.data().asDouble();
        final XYSeries s = new XYSeries(label);
        for( int j=0; j<xd.length; j++ ) s.add(xd[j],yd[j]);
        dataSet.addSeries(s);
    }

}


The output of the above code is this chart, in which you can see that punt distances increase as punter-hold times are longer.

![PuntDistance](https://i.imgur.com/BGkb6Pg.png)

**Rule Change Recommendation 1**

*Existing Rule*
* 9-1-2 : During a kick from scrimmage, only the end men (eligible receivers) on the line of scrimmage at the time of the snap, or an eligible receiver who is aligned or in motion behind the line and is more than one yard outside the end man, are permitted to advance more than one yard beyond the line before the ball is kicked.

*Rule Change*
* 9-1-2: During a kick from scrimmage, only the two most outer men on both horizontal sides of the ball (4 men total) on the line of scrimmage are considered eligible receivers at the time of the snap, are permitted to advance more than one yard beyond the line before the ball is kicked. Any other player on the line of scrimmage is ineligible and designated as a blocking role, unable to proceed more than 1 yard downfield until the ball is kicked.

*Modification Result*
* Enables the use of the spread and shield punt formations, putting 4 effective eligible receivers on the line.
* Consistent blocking down field of these two additional eligible receivers (4 total) will reduce their velocity of producing a damaging block/hit.
* Figure 3 and Figure 4 in the PPT Presentation show the Shield and Spread Punt formations that would now be acceptable.



**Rule Change Recommendation 2**

*Punt formation Rule*
* There is currently no rule for punt formation, so the recommendation is to create a rule that enables more fair catches.

*New Rule*
* For a punt kick from any formation (traditional, shield or spread), the punter may be no more than 10 yards behind the line of scrimmage.

*Rule Rationale*
* The data currently shows us that the longer the punts, the greater number of concussions are likely to occur.
* The data currently shows us that punts with fair catches yield fewer concussions.
* The data currently shows us that punts from the 5 yard line and in have resulted in 0 concussions.  This is because of the compressed distance of the punter from the line of scrimmage and the incentive to quickly kick the ball.

*Rule Result*
* Fewer steps for the punter to walk into the punt (videos show between 3-4 steps walk up) will yield reduced punt yardage. (# of punter steps were not part of the dataset)
* In combination with the Rule 1 modification suggestion, 4 punt team members will be closer to the punt returner.
* Ultimately more fair catches is the desired result.